# 🔹 Разработка и дообучение LLM‑ассистента для генерации текстовых рецептов на основе базы рецептов (Unsloth + TRL SFT)

### Построим прототип ассистента, который получает на вход только название блюда, автоматически находит соответствующую карточку рецепта в локальной базе (JSON), после чего генерирует связный, читабельный, структурированный текст рецепта (в стиле помощника), при необходимости добавляя полезные улучшения (советы, вариации, ошибки новичков).

## Исходные данные (входы и выходы)
База рецептов: recipes_db.json

Формат записи:

dish (string) — название блюда.

recipe (object):

servings (int) — количество порций.

time_minutes (int) — время приготовления.

ingredients (list[object]) — список ингредиентов:

name (string)

amount_g (int)

steps (list[string]) — шаги приготовления.

# 1. 🔹 Импорты и загрузка нужных инструментов

In [14]:
import torch

In [2]:
!pip -q uninstall -y torchaudio

In [3]:
!pip -q install -U unsloth trl transformers datasets accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 146.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
!pip -q uninstall -y torchaudio
!pip -q install -U unsloth trl transformers datasets accelerate bitsandbytes

# 2. 🔹 Создадим базу данных в коде (можно также использовать уже готовый файл)

In [12]:
import json

db = [
  {
    "dish": "Syrniki (Cottage Cheese Pancakes)",
    "recipe": {
      "servings": 2,
      "time_minutes": 25,
      "ingredients": [
        { "name": "Cottage cheese (9%)", "amount_g": 400 },
        { "name": "Egg", "amount_g": 50 },
        { "name": "Sugar", "amount_g": 20 },
        { "name": "Flour", "amount_g": 60 },
        { "name": "Salt", "amount_g": 2 },
        { "name": "Oil (for frying)", "amount_g": 15 }
      ],
      "steps": [
        "Mix cottage cheese, egg, sugar, and salt until smooth.",
        "Add flour and mix into a soft dough.",
        "Shape small patties; lightly dust with flour if needed.",
        "Fry on medium heat for 3–4 minutes per side until golden."
      ]
    }
  },
  {
    "dish": "Banana Oatmeal",
    "recipe": {
      "servings": 1,
      "time_minutes": 10,
      "ingredients": [
        { "name": "Rolled oats", "amount_g": 50 },
        { "name": "Milk", "amount_g": 200 },
        { "name": "Banana", "amount_g": 120 },
        { "name": "Honey", "amount_g": 10 },
        { "name": "Salt", "amount_g": 1 }
      ],
      "steps": [
        "Warm the milk in a small pot until hot (do not boil aggressively).",
        "Stir in oats and a pinch of salt; cook 3–5 minutes, stirring.",
        "Remove from heat and fold in sliced banana.",
        "Drizzle with honey and serve."
      ]
    }
  },
  {
    "dish": "Tomato Pasta (Simple Marinara)",
    "recipe": {
      "servings": 2,
      "time_minutes": 25,
      "ingredients": [
        { "name": "Dry pasta", "amount_g": 180 },
        { "name": "Canned tomatoes (crushed)", "amount_g": 350 },
        { "name": "Garlic", "amount_g": 6 },
        { "name": "Olive oil", "amount_g": 15 },
        { "name": "Salt", "amount_g": 3 },
        { "name": "Dried Italian herbs", "amount_g": 2 }
      ],
      "steps": [
        "Boil pasta in salted water until al dente.",
        "Warm olive oil, add garlic, and cook 30 seconds until fragrant.",
        "Add tomatoes, salt, and herbs; simmer 10–12 minutes.",
        "Toss pasta with sauce and serve."
      ]
    }
  },
  {
    "dish": "Greek Salad",
    "recipe": {
      "servings": 2,
      "time_minutes": 12,
      "ingredients": [
        { "name": "Cucumber", "amount_g": 180 },
        { "name": "Tomatoes", "amount_g": 220 },
        { "name": "Bell pepper", "amount_g": 120 },
        { "name": "Kalamata olives", "amount_g": 60 },
        { "name": "Feta cheese", "amount_g": 120 },
        { "name": "Olive oil", "amount_g": 15 },
        { "name": "Salt", "amount_g": 2 }
      ],
      "steps": [
        "Chop cucumber, tomatoes, and bell pepper into bite-size pieces.",
        "Add olives and crumble feta on top.",
        "Drizzle with olive oil, add salt, and gently toss.",
        "Serve immediately."
      ]
    }
  },
  {
    "dish": "Shakshuka (Eggs in Tomato Sauce)",
    "recipe": {
      "servings": 2,
      "time_minutes": 25,
      "ingredients": [
        { "name": "Eggs", "amount_g": 120 },
        { "name": "Canned tomatoes", "amount_g": 300 },
        { "name": "Onion", "amount_g": 80 },
        { "name": "Bell pepper", "amount_g": 120 },
        { "name": "Olive oil", "amount_g": 15 },
        { "name": "Garlic", "amount_g": 6 },
        { "name": "Paprika", "amount_g": 2 },
        { "name": "Salt", "amount_g": 2 }
      ],
      "steps": [
        "Sauté onion and bell pepper in olive oil for 5–7 minutes until soft.",
        "Add garlic and paprika; cook 30 seconds.",
        "Stir in tomatoes and simmer 8–10 minutes until slightly thickened.",
        "Make wells, crack in eggs, cover, and cook 4–6 minutes to desired doneness."
      ]
    }
  }
]


with open("recipes_db.json", "w", encoding="utf-8") as f:
    json.dump(db, f, ensure_ascii=False, indent=2)

#3. 🔹 Сборка обучающего датасэта из базы рецептов train.json (тк в нашем случае мы получаем на вход датасэт без нужного формата, если есть нужный формат, то этап можно пропустить)

 - Конвертация “структура → текст”: берем recipe (структуру с полями ingredients/steps/time/servings) и превращаем её в единый  шаблон out.

 - Формирование пары SFT: train.append({"input": inp, "output": out}) — это создание supervised примера. input тут является “контекстом” (название + карточка рецепта), output — эталонный ответ в нужном стиле.


In [11]:
import json

with open("recipes_db.json", "r", encoding="utf-8") as f:
    db = json.load(f)

train = []
for item in db:
    # dish + базовый рецепт
    inp = {"dish": item["dish"], "recipe": item["recipe"]}

    # целевой ответ (потом модель будет улучшать сама)
    r = item["recipe"]
    ingredients_lines = "\n".join([f"- {x['name']}: {x['amount_g']} г" for x in r["ingredients"]])
    steps_lines = "\n".join([f"{i+1}) {s}" for i, s in enumerate(r["steps"])])

    out = (
        f"{item['dish']} — отличный домашний вариант.\n\n"
        f"Время: {r['time_minutes']} минут. Порции: {r['servings']}.\n\n"
        f"Ингредиенты:\n{ingredients_lines}\n\n"
        f"Шаги:\n{steps_lines}\n\n"
        f"Советы:\n"
        f"- Если масса слишком влажная, добавь 1–2 ст. л. муки или манки и дай постоять 5 минут.\n"
        f"- Для более нежной текстуры протри творог через сито.\n"
        f"- Подавай со сметаной, ягодами или мёдом."
    )

    train.append({"input": inp, "output": out})

with open("train.json", "w", encoding="utf-8") as f:
    json.dump(train, f, ensure_ascii=False, indent=2)


# 🔹 4. Подготовка обучающих данных в нужный формат для SFTTrainer (модель должна видеть один текст, где есть и запрос пользователя, и ответ ассистента).

- Импортируем json  и Dataset из datasets
- Открываем train.json и загружаем его в raw: это список примеров, где каждый пример имеет поля input и output.
- Задаём EOS = "<|endoftext|>" — маркер конца ответа; добавляем его, чтобы модель училась понимать, где заканчивается ответ ассистента и не продолжала бесконечно.
- Описываем функцию to_text(ex), которая:
  - берёт ex["input"] (JSON-контекст блюда) и превращаем его в строку через json.dumps(..., ensure_ascii=False),
  - собирает один общий текст в формате диалога

- Создаём dataset = Dataset.from_list(raw) — превращаем список Python-словарей в датасет Hugging Face.
- Применяем .map(to_text) — добавляем каждому примеру новое поле text (или заменяем структуру так, чтобы дальше обучаться именно по нему).


In [8]:
import json
from datasets import Dataset

with open("train.json", "r", encoding="utf-8") as f:
    raw = json.load(f)

EOS = "<|endoftext|>"

def to_text(ex):
    user_json = json.dumps(ex["input"], ensure_ascii=False)
    return {"text": f"User: {user_json}\nAssistant: {ex['output'].strip()}{EOS}"}

dataset = Dataset.from_list(raw).map(to_text)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

# 🔹 5. Загрузка модели и подключение LoRA (Unsloth)

- Импортируем FastLanguageModel и is_bfloat16_supported: первый нужен для быстрой загрузки/ускорения модели и подготовки к дообучению, второй — чтобы позже правильно выбрать точность обучения (bf16/fp16).

- Задаём model_name и max_seq_length: выбираем небольшую instruct-модель (3B) и ограничиваем контекст до 1024 токенов, чтобы стабильнее помещалось на T4. (в перспективе использовать 2048)

- Загружаем модель и токенизатор через FastLanguageModel.from_pretrained(...): включаем load_in_4bit=True, чтобы уменьшить потребление памяти

- Подключаем LoRA через FastLanguageModel.get_peft_model(...): добавляем небольшие обучаемые адаптеры (а не переучиваем все веса), выбираем какие слои адаптировать (target_modules), и включаем экономию памяти через use_gradient_checkpointing="unsloth"


| Модель (пример тега) | Параметры | Размер (ГБ) | Для чего лучше всего |
|---|---:|---:|---|
| Llama 3.2 (`llama3.2:1b`) | 1B | 1.3 | Максимально быстрая и лёгкая, базовый “ассистент” и форматирование |
| Llama 3.2 (`llama3.2`) | 3B | 2.0 | Лучший компактный универсал, хорошо следует инструкциям |
| Gemma 2 (`gemma2:2b`) | 2B | 1.6 | Сильная маленькая модель, хорошее качество на малом размере |
| Gemma 2 (`gemma2`) | 9B | 5.5 | Более качественный текст и понимание, нужны больше ресурсы |
| Mistral (`mistral`) | 7B | 4.1 | Надёжная “середина” 7B для общих задач |
| Phi 3 Mini (`phi3`) | 3.8B | 2.3 | Компактная, но достаточно умная, хороший компромисс |
| Phi 3 Medium (`phi3:medium`) | 14B | 7.9 | Существенно сильнее, но заметно тяжелее по ресурсам |
| Gemma 2 (`gemma2:27b`) | 27B | 16.0 | Высокое качество, требует мощного железа |



In [9]:
from unsloth import FastLanguageModel, is_bfloat16_supported

model_name = "unsloth/Llama-3.2-3B-Instruct"
max_seq_length = 1024 # максимальная длина контекста (2048 - 4096)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=True, # Если мощный ПК можно попробовать 8‑бит или 16‑бит (если хочется чуть стабильнее качество и меньше артефактов)
    dtype=None,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16, # Больше r  -> потенциально лучше адаптация, но больше память/риск переобучения (8 - 64)
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    use_gradient_checkpointing="unsloth",# экономия памяти (можно отключить)
    random_state=3407, # воспроизводимость
)


==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2026.1.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# 🔹 6. Настройки и запуск SFT

 - запускаем supervised fine-tuning: берем готовую базовую модель (с подключёнными LoRA‑адаптерами) и дообучают её на датасете dataset, где в поле text уже лежат примеры в виде “User: … / Assistant: …”.

 -  создаем объект тренера SFTTrainer, который связывает вместе модель, токенизатор и обучающие данные, а затем командой trainer.train() выполняем само обучение — модель подстраивается так, чтобы на похожий вход выдавать ответ в нужном стиле и формате.

In [15]:
from transformers import TrainingArguments
from trl import SFTTrainer

args = TrainingArguments(
    per_device_train_batch_size=2, # можно (4-32)
    gradient_accumulation_steps=4, # можно (1 - 16)
    warmup_steps=10, #(1 - 5% от max_steps)
    max_steps=120, # (на больших дс до 1000+)
    learning_rate=2e-4, #скорость обучения
    logging_steps=1, #чем больше, тем чище вывод (1 - 50)
    optim="adamw_8bit", #Если ПК мощнее можно оставить (бывает просто выгоднее по памяти), либо перейти на обычный AdamW, если память не проблема
    weight_decay=0.01, #(0 - 0.1) регуляризация
    lr_scheduler_type="linear",
    fp16=not is_bfloat16_supported(), #Если мощная GPU с bf16 (часто A100/4090 и т.п.)
    bf16=is_bfloat16_supported(),
    output_dir="outputs",
    seed=3407,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
    args=args,
)

trainer.train()



Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 120 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
1,0.006000
2,0.005900
3,0.006200
4,0.005500
5,0.005300
6,0.004900
7,0.004600
8,0.004400
9,0.004300
10,0.004400


TrainOutput(global_step=120, training_loss=0.004033690291301658, metrics={'train_runtime': 374.7665, 'train_samples_per_second': 2.562, 'train_steps_per_second': 0.32, 'total_flos': 4431149772152832.0, 'train_loss': 0.004033690291301658, 'epoch': 120.0})

# 🔹 7. Подготовка к ответу + ответ

In [16]:
import difflib, json # difflib нужен для похожего совпадения названия (если пользователь написал с ошибкой/вариантом).
from unsloth import FastLanguageModel

# загрузка базы данных
with open("recipes_db.json", "r", encoding="utf-8") as f:
    db = json.load(f)
# чтение списка карточек из файла
index = {x["dish"].strip().lower(): x for x in db}
all_names = list(index.keys()) # Делает словарь “название блюда → вся карточка”, чтобы искать быстро и без учета регистра/пробелов.

# поиск (1 - поиск по точному совпадению, 2 - ищет похожее, если нет, то None)
def find_recipe(dish_name: str, cutoff: float = 0.6):
    key = dish_name.strip().lower()
    if key in index:
        return index[key]
    matches = difflib.get_close_matches(key, all_names, n=1, cutoff=cutoff)
    if matches:
        return index[matches[0]]
    return None

FastLanguageModel.for_inference(model)


#Находит блюдо в базе.
#Формирует context = {"dish": ..., "recipe": ...} — то, что будет вставлено в промпт.
#Делает большой промпт с жесткими правилами (не менять граммовки, английский язык, не JSON и т.д.).
def agent_recipe(dish_name: str):
    item = find_recipe(dish_name)
    if item is None:
        return f"Не нашла блюдо «{dish_name}» в базе."

    context = {"dish": item["dish"], "recipe": item["recipe"]}

    prompt = f"""You are a cooking assistant. You are given a dish card in JSON.

HARD RULES (must follow):
- The dish title in your answer must be EXACTLY: "{item['dish']}".
- Keep the ingredients and gram amounts from the JSON as the base. Do not change numbers.
- Write in English only. Do NOT use Cyrillic or mixed alphabets.
- Do not output JSON.

TASK:
Write a clear, friendly, well-structured recipe using the JSON card. You MAY improve the recipe by adding:
- 2 variations: "healthier" and "festive" (what to change),
- 3 common beginner mistakes and how to avoid them,
- 1 serving suggestion.

JSON card:
{json.dumps(context, ensure_ascii=False, indent=2)}

RESPONSE FORMAT:
Title: ...
Short intro (2–3 sentences)
Ingredients (keep exact grams from JSON)
Steps (6–10 steps, paraphrase the JSON steps; do not copy verbatim)
Variations (healthier / festive)
Beginner mistakes (3 bullets)
Serving suggestion (1–2 sentences)
"""


    # подготовка входа для модели
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    out = model.generate( # генерация
        **inputs,
        max_new_tokens=450, #(64 - 1024)
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_p=0.85,
        repetition_penalty=1.1,
        no_repeat_ngram_size=4,
    )

    gen = out[0][inputs["input_ids"].shape[-1]:]
    text = tokenizer.decode(gen, skip_special_tokens=False)

    # убираем дубль
    text = text.split("<|endoftext|>")[0]

    # убираем возможные спец-токены
    return text.replace("<|begin_of_text|>", "").strip()

print(agent_recipe("Greek Salad"))


—- —- —-

Title: Greek Salad

A classic summer salad made with fresh vegetables, tangy feta, and a drizzle of olive oil.

Ingredients:
- Cucumber: 180 g
- Tomatoes: 220 g
- Bell pepper: 120 g
- Kalamata olIVES: 60 g
- Feta cheese: 120 г
- Olive oil: 15 г
- Salt: 2 г

Steps:
1) Chop cucumber, tomatoes, and bell pepperinto bite-size pieces.
2) Add olives and crackle feta on t.
3) Drizzle with olive oil, add salt, and gently toss.
4) Slice an avocado (optional) and serve alongside.
5) If mixture is too dry, add 1–2 tablespoons of water or olive oil.
6) Cover and let sit for 5 minutes before serving.

Variations:
Healthier: Replace feta with part-mashed banana or applesauce for a sweeter twist.
Festive: Stir in 1/4 cup of chopped fresh berries (such as strawberries or blueberries) and 1 tablespoon of honey.

Beginner mistakes:
• Do not overchop ingredients, as this can make the salad too soft.
• Fail to drizzle with oliveoil just before serving, as it can become soggy if left for too long.